# Différents algorithmes importants 


### LeNet

La classification est un problème du traitement d'images où l'algorithme va reconnaitre la principale composante de l'image. 

C'est cette catégorie de problèmes que voulait traiter <a href==http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf> le réseau proposé par Yann Lecun </a>. Une des application était de résoudre le défi MNIST : soit la classification des nombres `0` à `9` écrits à la main.


![leNet](http://tecmemo.wpblog.jp/wp-content/uploads/2017/03/dl_lenet-02-644x240.png)

L'architecture utilisé est maintenant une architecture incontournable. 

Le réseau est consituté de 2 couches de convolution, couplées avec des couches de pooling.
Les 3 dernières couches sont des couches de neurones densément connectés qui permettent de conclure sur la classe la plus présente dans l'image.

### FCN

Le problème de classification a rapidement été complété par le problème de segmentation sémantique. Le but est alors de détecter les différents objets, les segmenter, puis les identifier. 

![semanticSegmentation](img/semanticSegmentation.png)

LeNet devient alors insuffisant. Il est en effet incapable d'associer à chaque pixel une classe. Il faut donc changer de paradigme.

Un des premiers réseaux à apporter une solution était le <a href=https://arxiv.org/pdf/1411.4038.pdf> Fully Convolution Network </a> (FCN).

Ce dernier apportait plusieurs contibutions. 

Tout d'abord l'idée est de remplacer la couche finale densément connectée du réseau, par une couche convolutionnelle. Cela permet d'obtenir une carte de résultat et plus seulement une catégorie. 

#### ![FCNimage](http://blog.qure.ai/assets/images/segmentation-review/FCN%20-%20illustration.png)

Mais pour obtenir des cartes de résultat pertinentes, il faut résoudre le problème de la transformation au cours des convolutions.


Nous avons vu en effet que les couches de convolution transformaient l'image initiale en un contenu de plus haut niveau mais moins bien localisé. Plus concrétement, le réseau va traiter un ensemble de pixels qui définisse un chat pour en déduire la présence d'un objet chat. Cet objet est une analyse haut niveau de l'image mais sa place est inconnue.

Cette caractrétique des CNN découle de la pyramide des 'field of view'. 
Les FCN éliminent ce problème grâce à **l'upsampling**.

Une première approche pour augmenter la résolution d'une image est d'interpoler les pixels voisins. Le pixel rajouté est alors une somme pondéré des pixels voisins.
C'est cette même approche qui est utilisé ici. A la différence que c'est le réseau même qui apprendra à réaliser l'interpolation au travers de couches particulières, appelées **deconvolutions layers**.
Une telle structure permet notamment au réseau d'apprendre les interpolations non linéaires.

### U-Net

Le [U-Net](https://arxiv.org/pdf/1505.04597.pdf) se penche sur le même problème que les **FCN** avec une amélioration significative sur le processus de **upsampling**.

L'augmentation de la résolution s'opère ici d'une part grâce aux **deconvolutions layers** mais aussi grâce aux **coarse connections**. 



![UNetimage](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png)

Le réseau se répartit désormais en 2 parties.

La branche gauche du réseau réalise des opérations de convolutions classiques. Celles-ci vont permettre d'extraire de l'image les composantes sémantiques; *sans* information de localisation.

Le rôle de la branche de droite est de resituer sur une image de dimension approximativement identique à l'image de départ, l'emplacement des différents objets. 
Ce travail sera réalisé par des couches de déconvolution, comme précédemment. 

La nouveauté consiste dans le fait que chaque couche de déconvolution est suivie d'une couche de convolution classique, fusionnat :
- le résultat interpolé de la couche précédente 
- le résultat issue de la couche symétrique sur le coté gauche

Ce phénomène est notamment illustré par l'image précédente

On observe tout au long de la partie gauche une modification de la taille des tenseurs. Intiallement de taille `572x572x1`, ces derniers acquièrent à la fin une taille de `28x28x1024` pour le tenseur $T_{0}$.

On observe bien ici l'apparition des élèments sémantiques au détriment de l'information de localisation.



$T_{0}$ est alors interpolé en un tenseur $T_{1}$ de dimensions `56x56x512`. On s'attend donc à ce que la couche d'interpolation replace plus finement les informations.

Toutefois, pour corriger son travail, nous allons fusionner $T_{1}$ avec le résultat du niveau symétrique de la branche gauche, le tenseur $T_{-1}$. $T_{-1}$ porte l'information que le réseau a extrait à la résolution `56x56`. 

Nous obtenons alors un tenseur de dimensiosn ` 1024x56x65` qui sera transformé après deux convolutions en un tenseur `512x52x52`, mêlant au mieux les informations locales à cette résolution et les informations sémantiques.

Répeter ce processus permettra de reconstituer une image finale segmentée sémantiquement.

### CPM

Nous pouvons à présent nous pencher sur un nouveau problème de traitement d'images : la labelisation. Ce dernier désigne la tâche de détecter et placer précisément un élèment au sein d'une image. 

![CPMgif](https://s1.qwant.com/thumbr/0x0/3/2/aaec6e4ea36ca57ed4190262087e04/b_1_q_0_p_0.jpg?u=http%3A%2F%2Fliris.cnrs.fr%2Fchristian.wolf%2Fgraphics%2Fanimated_handposejoints.gif&q=0&b=1&p=0&a=1)

Dans l'exemple ci-dessus, il est par exemple question de placer correctement les différentes parties doigts de la main.

Ce problème est intimement lié au problème de segmentation sémantique. Il est également nécessaire de concilier compréhension sémantique de l'image et conservation de la taille de l'image.

Mais nous soulevons ici une série de nouvelles problématiques. L'algorithme doit en effet être capable de gérer la présence et l'abscence des différents points à détecter ainsi que d'être capable de situer un élément vis à vis des autres.

En effet, si nous souhaitons différencier la main droite, de la main gauche, il est nécessaire que l'algorithme replace ce point vis à vis du corps entier.

Les réseaux que nous avons vu précédemment, en particulier U-Net peut aborder ce problème. Toutefois l'algorithme des [Convolutional Pose Machine ](https://arxiv.org/pdf/1602.00134.pdf) a particulièrement brillé en 2016 pour résoudre ce problème.

Cet algorithme s'appuie simultanément sur des réseaux de convolutions FCN et sur la démarche des pose machine.


Les **pose machines** sont un processus itératif. L'idée centrale est de combiner un algorithme de reconnaissance locale des images, avec un algorithme de compréhension globale des images.

A l'étape $0$, l'algorithme local essaie de reconnaitre les différentes composantes de l'image. Il est alors difficile pour l'algorithme de placer correctement les élèments cachés entièrement ou partiellement. Il a également des difficultés à distinguer les objets similaires : poignet droit, poignet gauche.

A l'étape $1$, l'algorithme peut analyser à nouveau les images au niveau local, mais en disposant cette fois des résultats obtenus à l'étape $0$.

L'algorithme a alors la capacité de corréler les indices locaux pour un membre avec la présence des autres.

![CPMimage](https://github.com/shihenw/convolutional-pose-machines-release/raw/master/testing/python/figures/teaser-github.png)

Par exemple, on peut s'attendre à ce que l'algorithme s'aide du fait qu'il est sur d'avoir trouvé le coude droit pour en déduire l'avant bras droit et le poignet droit. 

Un innovation importante des Convolutional Pose Machine est le fait que la fonction de corrélation, qui permet au programme d'acquérir une vision globale du problème est apprise. Le réseau de neurones est donc libre de reconnaitre les différents membres du corps aussi bien au niveau local qu'au niveau global. 

Une représentation de ce réseau est visible ci-desssous.

![imageCPM](http://openresearch.ai/uploads/default/optimized/1X/18f20fd72702bd2157277019938753e79bc6b5fb_1_690x414.jpg)

Ce réseau est composé de 2 parties : 
- Une partie $P_{1}$ consacrée à l'étape initiale, réalisé une seule fois 
- Une partie $P_{n}$ consacrée à la partie itérative, pouvant être répétée un nombre arbitraire de fois

$P_{1}$ prend en entrée une image classique, de dimension `h x w x 3`. Cette image est analysée par un FCN classique. Ce dernier fournit donc en sortie un tenseur de dimension `h' x w' x P` où `P` est le nombre d'élèments à identifier sur l'image.
Par conséquent, pour chaque élèment, nous disposons d'une carte de probabilité de présence sur l'image.

Il est important de remarquer la présence d'une fonction de coût $f_{1}$ à la fin de $P_{1}$. Cette fonction de coût permet notamment d'échapper au problème du *vanishing gradient* qui survient lorsque la taille du réseau est trop importante.

$P_{n}$ est composé des 2 branches : 
- une branche qui va amener les cartes de probabilité de la partie précédente 
- une branche qui va retraiter les images initiales
Ces deux branches fusionnent leurs résultats. On obtient alors un tenseur mêlant information purement local et information global. Ce tenseur est alors traité par un réseau convolutionnel spécifique. La sortie est alors une carte de probabilité.

Celle-ci peut alors à son tour participer à nourrir une nouvelle étape de l'algorithme.

Nous nous attendons qu'à chaque nouvelle étape permette à l'algorithme de gagner en compréhension. Comme toujours, il faut néanmoins veiller à ne pas faire trop d'étapes, afin de ne pas gêner les capacités de généralisation du réseau.